In [22]:
# Library import

import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import pandas as pd
from pathlib import Path
import datetime as dt
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

delta=dt.timedelta(days=1)

In [23]:
# Get main path and other folders
s_mainpath=Path.cwd()
print(s_mainpath)
s_parentpath=s_mainpath.parent.parent.parent
print(s_parentpath)

c:\Alejo\Eje de Planeación\Análisis Eléctrico\ScriptsPython
c:\Alejo


In [24]:
# Lectura de datos de enrtrada para el cálculo
s_parentpath=s_mainpath
filepath=s_parentpath.joinpath(s_parentpath,'ArchivosEntrada','IPOEM.xlsx')

# Caribe 2 1300-1400
sheet_name='Caribe2'
skip_rows = 1
start_col=0
end_col = 2
end_row = 12
df_UniCar2_1=pd.read_excel(filepath, header=0,sheet_name=sheet_name, usecols=range(start_col, end_col+1),skiprows=skip_rows, nrows=end_row+1)

# Caribe 2 1200-1300
sheet_name='Caribe2'
skip_rows = 1
start_col=4
end_col = 6
end_row = 6
df_UniCar2_2=pd.read_excel(filepath, header=0,sheet_name=sheet_name, usecols=range(start_col, end_col+1),skiprows=skip_rows, nrows=end_row+1)
for col in df_UniCar2_2.columns:
    df_UniCar2_2.rename(columns={col:col[:-2]},inplace=True)

# Caribe 2 1300-1400 T3
sheet_name='Caribe2'
skip_rows = 1
start_col=8
end_col = 10
end_row = 16
df_UniCar2_1T3=pd.read_excel(filepath, header=0,sheet_name=sheet_name, usecols=range(start_col, end_col+1),skiprows=skip_rows, nrows=end_row+1)
for col in df_UniCar2_1T3.columns:
    df_UniCar2_1T3.rename(columns={col:col[:-2]},inplace=True)

#Unidades equivalnete
sheet_name='PesoUni'
skip_rows = 0
start_col=0
end_col = 4
end_row = 100
df_PesoUni_Car2=pd.read_excel(filepath, header=0,sheet_name=sheet_name, usecols=range(start_col, end_col+1), nrows=end_row+1) 

# Unidades Atlantico
sheet_name='Atlantico'
skip_rows = 0
start_col=0
end_col = 2
end_row = 30
df_UniAtl=pd.read_excel(filepath, header=0,sheet_name=sheet_name, usecols=range(start_col, end_col+1),skiprows=skip_rows, nrows=end_row+1)

# Caribe 2 1300-1400
sheet_name='Bolivar'
skip_rows = 0
start_col=0
end_col = 2
end_row = 30
df_UniBol=pd.read_excel(filepath, header=0,sheet_name=sheet_name, usecols=range(start_col, end_col+1),skiprows=skip_rows, nrows=end_row+1)

In [25]:
# Función para calcular las unidades según la demanda
def CalcularUnidades(Demanda,df_data,df_dataIni,Fecha):

    if Fecha==dt.datetime.strptime('2024-01-02','%Y-%m-%d'):
        stop=1
    if Fecha>=dt.datetime.strptime('2024-02-01','%Y-%m-%d'):
        rows,cols=df_data.shape

        if (Demanda>df_data.at[0,'ValMin']) and (Demanda<=df_data.at[rows-1,'ValMax']):

            Val=df_data[(Demanda>df_data.ValMin) & (Demanda<=df_data.ValMax)]['Unidades'].values[0]

        else:
            Val=0
    else:
        rows,cols=df_dataIni.shape

        if (Demanda>=df_dataIni.at[0,'ValMin']) and (Demanda<=df_dataIni.at[rows-1,'ValMax']):

            Val=df_dataIni[(Demanda>df_dataIni.ValMin) & (Demanda<=df_dataIni.ValMax)]['Unidades'].values[0]

        else:
            Val=0

    return Val

In [26]:
def ApplyFuncyionValue(df_Apply,df_data,df_dataIni):

    # df_Apply=df_Apply.set_index('fecha')

    l_row=[]
    for ind in df_Apply.index:
        #print(ind)
        
        x=df_Apply.at[ind,'Caribe2']
        hora=df_Apply.at[ind,'periodo']
        val=CalcularUnidades(x,df_data,df_dataIni,Fecha=pd.to_datetime('2024-09-06'))
        if ((hora>=1 and hora<=5) or (hora>=10 and hora<=14) or (hora>=17 and hora<=18)):
            val=val-0.5
        l_row.append(val)

    df_Results=pd.DataFrame(l_row,index=df_Apply.fecha)
    
    return df_Results

In [27]:
s_filepath=r'C:\Alejo\cops\Data\DatosEntradaCOPS_Base - copia.xlsx'
df=pd.read_excel(s_filepath, sheet_name='DemandaMod')

In [28]:
df.head(5)

,fecha,periodo,SubAntioquia,SubAtlantico,SubBolivar,SubCerromatoso,SubCordoba-Sucre,SubGCM,Caribe2,Caribe,SIN
0,2024-09-13,1,6298.404576,759.475931,605.616413,257.932057,918.651202,698.324847,2063.417191,3240.000450,9538.405026
1,2024-09-13,2,6082.072729,737.225267,589.804031,254.404320,886.522061,678.104605,2005.133903,3146.060284,9228.133012
2,2024-09-13,3,5925.677797,716.003692,575.185685,250.545849,858.040881,656.892473,1948.081850,3056.668580,8982.346377
3,2024-09-13,4,5844.322776,698.765053,562.079498,248.778228,833.864037,639.050979,1899.895530,2982.537794,8826.860570
4,2024-09-13,5,5914.526078,685.847871,548.801110,248.381817,815.233612,622.262757,1856.911739,2920.527168,8835.053247


In [29]:
df_UniCar2=ApplyFuncyionValue(df,df_UniCar2_1,df_UniCar2_1T3) # Unidades equivalentes a partir de la demanda Modificada

In [30]:
df_UniCar2

,0
fecha,
2024-09-13,5.5
2024-09-13,4.5
2024-09-13,4.5
2024-09-13,4.5
2024-09-13,4.0
...,...
2024-11-30,6.5
2024-11-30,6.5
2024-11-30,6.0
